In [1]:
!pip install -U openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
df = pd.read_csv("/content/Feedback.csv")
df.head()


,text,sentiment
0,You have selected some technologies for the as...,Critical
1,The work lacks novelty and presentation,Critical
2,This work does not reflect an understanding of...,Critical
3,"Learning is never done without error, so you s...",Critical
4,I would have implemented the method using this...,Critical


In [3]:
def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        json_response = '{"sentiment": "' + row['sentiment'] + '"}'
        fine_tuning_data.append({
            "messages": [
                {"role": "user", "content": row['text']},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data

dataset = pd.read_csv("Feedback.csv")
converted_data = convert_to_gpt35_format(dataset)
converted_data[0]['messages']


[{'role': 'user',
  'content': 'You have selected some technologies for the assessment but the assessment objective was to see how you can describe or justify each chosen technology. So you have failed to fulfil this objective.'},
 {'role': 'assistant', 'content': '{"sentiment": "Critical "}'}]

In [4]:
import json
json.loads(converted_data[0]['messages'][-1]['content'])

{'sentiment': 'Critical '}

In [5]:
from sklearn.model_selection import train_test_split

# Stratified splitting. Assuming 'sentiment' can be used for stratification
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    stratify=dataset['sentiment'],
    random_state=42  # for reproducibility
)

In [6]:
train_data[0]

{'messages': [{'role': 'user',
   'content': 'Learning is never done without error, so you should be spending more time.'},
  {'role': 'assistant', 'content': '{"sentiment": "Critical "}'}]}

In [22]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')

training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)


In [23]:
from openai import OpenAI
client = OpenAI(api_key = '# your OpenAiAPIKey') #Get you Open AI key

In [24]:
# Upload Training and Validation Files
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

print("Training file ID:",training_file.id)
print("validation file ID:",validation_file.id)


Training file ID: file-7qYNVKlGpyDt0hPVObmfcDb1
validation file ID: file-03ie2eQj9HjzaiL6uF6GgKaM


In [25]:
# Create Fine-Tuning Job
suffix_name = "project"

response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)
response

FineTuningJob(id='ftjob-AwPR4HJxqryAzn6zf0WsleKp', created_at=1714654650, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-H3UMkrdAxYQ2kGtPtGy9MHxS', result_files=[], seed=1090234613, status='validating_files', trained_tokens=None, training_file='file-7qYNVKlGpyDt0hPVObmfcDb1', validation_file='file-03ie2eQj9HjzaiL6uF6GgKaM', estimated_finish=None, integrations=[], user_provided_suffix='project')

In [26]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-AwPR4HJxqryAzn6zf0WsleKp', created_at=1714654650, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-H3UMkrdAxYQ2kGtPtGy9MHxS', result_files=[], seed=1090234613, status='validating_files', trained_tokens=None, training_file='file-7qYNVKlGpyDt0hPVObmfcDb1', validation_file='file-03ie2eQj9HjzaiL6uF6GgKaM', estimated_finish=None, integrations=[], user_provided_suffix='project'), FineTuningJob(id='ftjob-MZhE4PMk9EUFWtuYDqw8RFpP', created_at=1714654246, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:project:9KQFpvDo', finished_at=1714654507, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_

In [27]:
response = client.fine_tuning.jobs.retrieve("ftjob-HbdEbHvR6jKpPzJq8DEqUcgU")
response

FineTuningJob(id='ftjob-HbdEbHvR6jKpPzJq8DEqUcgU', created_at=1713450777, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:project:9FNCp1Uv', finished_at=1713451150, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-H3UMkrdAxYQ2kGtPtGy9MHxS', result_files=['file-zGnsae9qz9H9qGXtrPMtJ8Qu'], seed=257275051, status='succeeded', trained_tokens=2583, training_file='file-oRNiPTP3ofwUdElHPPpyadT9', validation_file='file-jDRj5h45GxIBsZTU18Dz3U5r', estimated_finish=None, integrations=[], user_provided_suffix='project')

In [28]:
fine_tuned_model_id = response.fine_tuned_model
print("\nFine-tuned model id:",fine_tuned_model_id)


Fine-tuned model id: ft:gpt-3.5-turbo-0125:personal:project:9FNCp1Uv


In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def format_test(row):
    formatted_message = [{"role": "user", "content": row['text']}]
    return formatted_message

def predict(test_messages, fine_tuned_model_id):
    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
    )
    return response.choices[0].message.content




In [19]:
def store_predictions(test_df, fine_tuned_model_id):
    test_df['Prediction'] = None
    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        test_df.at[index, 'Prediction'] = prediction_result

    test_df.to_csv("predictions.csv")


In [30]:
test_df = pd.read_csv("/content/test.csv")
store_predictions(test_df, fine_tuned_model_id)
